In [21]:
import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
from typing import Dict
import logging
logging.getLogger().setLevel(logging.INFO)

import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [2]:
filepath = '../data/US_latest_data_20220428.parquet' 
return_filepath = '../data/Exp_Return_Mom.parquet'
df = pd.read_parquet(filepath, engine = 'pyarrow')
expected_return = pd.read_parquet(return_filepath, engine = 'pyarrow')

In [3]:
def clean_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    # select required months
    df = df[["DateYYYYMM","ws_id","Price","Div","mom","mktcap","median_volume_usd"]]
    df = df.rename(columns={"DateYYYYMM": "date", "ws_id":"asset", "Price":"price", "Div":"div",
           "mom":"factor"})
    
    # Correct datetime 
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    # Change units for dollar valued data
    df['mktcap'] = df['mktcap'] *1000000
    df['median_volume_usd'] = df['median_volume_usd'] *1000000
    
    # replace missing values
    df['div'] = df['div'].fillna(0)
    df['price'] = df['price'].dropna()
    
    return df

def clean_return_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"Date": "date", "Asset":"asset", "Exp_Return":"exp_return"})
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    return df

In [4]:
df = clean_stock_data(df)
expected_return = clean_return_data(expected_return)

In [5]:
def filter_eligible_investments(df: pd.DataFrame, date: pd.Timestamp, ratio: float = 0.25, size: int = 1000) -> list:
    """
    Return the dataframe containing the top ratio% assets based on the factor of the top 1000 assets  of top asset based on market cap.
    Remove assets that no longer exist 1 period ahead
    """
    factor_num = int(ratio*size)
    
    df_current = df[df['date'] == date]
    df_next = df[df['date'] == date + pd.DateOffset(months=1)]
    
    asset_diff = set(df_current['asset']) - set(df_next['asset'])
    df_current = df_current[~df_current['asset'].isin(asset_diff)]
    
    df_by_mktcap = df_current.sort_values(by = 'mktcap', ascending=False).reset_index(drop=True)[:size]
    df_by_factor = df_by_mktcap.sort_values(by = 'factor', ascending=False).reset_index(drop=True)[:factor_num] 
     
    return df_by_factor.asset.to_list()

def update_assets(df: pd.DataFrame, valuation: float) -> Dict:
    """
    Return the dictionary of asset_names as the key and price/quantity tuple as the value. The proportion how much to purchase is determined by the  
    """
    
    df['ratio'] = df['mktcap']/df['mktcap'].sum()
    df['quantity'] = np.floor(df['ratio']*valuation/df['price'])
    assets = dict(zip(df['asset'], zip(df['quantity'], df['price'])))
    return assets
    

In [36]:
no_cost_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 10e8, # CASH+SHARES*PRICE + DIV
        'cash': 10e8,
        'div_paid': 0, #keep for debugging
        # 'index': 1, #Will be added back in if we consider cash injections
        'asset': {}, # {asset: (quantity,price)},
        'turnover': 0,
        'total_cost_explicit': 0,
        'total_cost_implicit': 0,
        # 'cash_flow': 0
    },
}


for date in pd.to_datetime(df['date'].sort_values().unique())[:10]:
    # filter the dataframe to the current date 
    df_current = df[df['date'] == date]
    
    # initialise current date key in portfolio
    no_cost_portfolio[date] = {}
    
    # update dividents paid
    prev_assets = no_cost_portfolio[date-pd.DateOffset(months=1)]['asset'] 
    
    div_paid = 0
    asset_value = 0
    for asset_name, quantity_price in prev_assets.items():
        if asset_name not in df_current.asset.to_list():
            asset_value += quantity_price[0]*quantity_price[1]
        else:
            asset_value += float(quantity_price[0] * df_current.loc[df_current['asset'] == asset_name, 'price'])
            div_paid += float(quantity_price[0]*df_current.loc[df_current['asset'] == asset_name, 'div'])

    
    no_cost_portfolio[date]['div_paid'] = div_paid
    no_cost_portfolio[date]['valuation'] = asset_value + div_paid + no_cost_portfolio[date - pd.DateOffset(months=1)]['cash'] 
    
    # print(f"{date}: {no_cost_portfolio[date]['valuation']}")
    
    # subset to eligible assets names
    asset_list = filter_eligible_investments(df, date, ratio, size)
    df_current = df[(df['date'] == date) | (df['asset'].isin(asset_list))]
    
    # update the assets to the preferred ones
    no_cost_portfolio[date]['asset'] = update_assets(df_current, no_cost_portfolio[date]['valuation'])
    
    # update cash amount
    no_cost_portfolio[date]['cash'] = no_cost_portfolio[date]['valuation'] - sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in no_cost_portfolio[date]['asset'].items()])

    

In [38]:
no_cost_portfolio[pd.to_datetime("197210", format = "%Y%m")]['valuation']#["b'51086'"][0]#['valuation'] # valutation at dec 2020 should be 70037236032 a return of 14.28% calculate by (val_end/val_1)^(12/n)

955339739.9256467

## Optimisation 

#### Data:
* $A$: set of assets = (the set of assets we are aiming to trade) $\cup$ (the set of assets we currently own)


* $m_j$ = expected return for asset $j\in A$ 
* $u_j$ = implicit cost lower bound for trading large proportions of asset $j\in A$   
* $v_j$ = implicit cost lower bound for trading small proportions of asset $j\in A$   
* $s$ = explicit cost lower bound for trading large proportions of asset $j\in A$   
* $t$ = explicit cost lower bound for trading small proportions of asset $j\in A$   
* $a_j$ = pre-trade proportion of the portfolio made up by asset $j\in A$ (this value is zero if we do not currently own it)
* $b_j$ = 1 if the asset $j\in A$ is in the top 250 momentum assets and 0 otherwise.
* $e$ = small value 
* $M$ = large value


#### Variables:
* $w_j$: weight of trade for stock $j\in A$
* $c_{Ej}$: explicit cost of trading asset $j\in A$ 
* $c_{Ij}$: implicit cost of trading asset $j\in A$ 
* $x_j$: binary variable, taking value $1$ if the weight of trade for stock $j\in A$ is $0$ and $1$ otherwise.

#### Model:

Max $z = \sum_{j\in A} m_j w_j - c_{Ej} - c_{Ij}$

subject to 
* $c_{Ej} \geq u_j w_j^2, \quad \forall j\in A$ 
* $c_{Ej} \geq v_j w_j, \quad \forall j\in A$ 
* $c_{Ej} \geq -v_j w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq -s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq t x_j, \quad \forall j\in A$ 
* $w_j \geq e - M(1-x_j), \quad \forall j \in A$
* $w_j \leq -e + M x_j, \quad \forall j \in A$
* $\sum_{j\in A} w_j = 0$
* $w_j \geq a_j, \quad \forall j \in A$
* $w_j \leq b_j, \quad \forall j \in A$
* $w_j \in [-1,1], x_j \in \{0,1\}, c_{Ij}, c_{Ej} \geq 0, \quad \forall j \in A$
 

In [ ]:
def calculate_performance(portfolio: Dict) -> float:
    
    min_date = min(portfolio.keys()) + pd.DateOffset(months=1)
    max_date = max(portfolio.keys())
    n = len(portfolio.keys())
    
    P = (portfolio[max_date]['valuation']/portfolio[min_date]['valuation'])**(12/n) -1
    
    return P

def calculate_mean_trade_cost(portfolio: Dict) -> float:
    
    n = len(portfolio.keys())
    c = 0
    for monthly_portfolio in list(portfolio.values())[1:]:
        total_cost = monthly_portfolio['total_cost_explicit'] + monthly_portfolio['total_cost_explicit']
        valuation = monthly_portfolio['valuation']
        
        c += total_cost/valuation
    
    P = (12/n)*c
    
    return P


def calculate_average_turnover(portfolio: Dict) -> float:
    n = len(portfolio.keys())
    c = 0
    for monthly_portfolio in list(portfolio.values())[1:]:
        c += monthly_portfolio['turnover']
    
    P = (1/n)*c
    
    return P
        

def value_assets_and_dividends(prev_assets: Dict, df_current:pd.DataFrame) -> tuple:
    div_paid = 0
    asset_value = 0
    for asset_name, quantity_price in prev_assets.items():
        if asset_name not in df_current.asset.to_list():
            asset_value += quantity_price[0]*quantity_price[1]
        else:
            asset_value += float(quantity_price[0] * df_current.loc[df_current['asset'] == asset_name, 'price'])
            div_paid += float(quantity_price[0]*df_current.loc[df_current['asset'] == asset_name, 'div'])
    
    return div_paid, asset_value

def calculate_large_implicit_cost(monthly_portfolio: Dict, df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula
    6*valuation/market_cap
    This is the implicit cost used for larger transactions.
    """
    df_monthly.loc[df_monthly['mktcap'] ==0, 'mktcap'] = np.inf # should the cost be zero if there's no market cap or should it be large?
    valuation = monthly_portfolio['valuation']
    df_monthly.loc[:,'large_implicit_cost'] = 6*valuation/df_monthly['mktcap']   
    
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['large_implicit_cost']))
    
    return implicit_cost


def calculate_small_implicit_cost(df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula,
    bid_ask_spread/valuation
    This is the implicit cost used for smaller transactions
    """  
    df_monthly.loc[:,'small_implicit_cost'] = 0.0013 * np.sqrt(np.median(df_monthly['mktcap'])/df_monthly['mktcap'])
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['small_implicit_cost']))
    
    return implicit_cost    


def assemble_monthly_data(df:pd.DataFrame, expected_return:pd.DataFrame, target_portfolio: Dict, date:pd.Timestamp, ratio: float = 0.25, size: int = 1000 ) -> pd.DataFrame:
    
    df_current = df[df.date == date]
    monthly_return = expected_return[expected_return['date'] == date]
    df_current = pd.merge(df_current, monthly_return, on = ['date', 'asset'], how="outer").fillna(0)
    
    buy_list = filter_eligible_investments(df, date, ratio, size)
    df_current.loc[df_current['asset'].isin(buy_list), 'buy'] = 1
    df_current['buy'] = df_current['buy'].fillna(0)
    
    
    portfolio_assets = pd.DataFrame.from_dict(target_portfolio[date]['asset'], orient='index')
    portfolio_assets = portfolio_assets.reset_index().rename(columns={'index':'asset'})
    df_current = pd.merge(df_current, portfolio_assets, on = ['asset'], how="outer").fillna(0)
    if date == pd.to_datetime("197201", format = "%Y%m"):
        df_current.loc[df_current['asset'] == 'sell_asset', 'price'] = 1
        # df_current['sell'] = - df_current[0] * df_current[1] / target_portfolio[date]['valuation']
        # print(df_current['sell'])
    # else:
    df_current['sell'] = - df_current[0] * df_current['price'] / target_portfolio[date]['valuation']
    # df_current['sell'] = - df_current[0] * df_current['price'] / target_portfolio[date]['valuation']
    # df_current.to_csv(f"../data/df_current_{date}.csv", index = False) 
    # print(f"portfolio_assets: {df_current}")
    
    forced_sale_list = list(set(target_portfolio[date]['asset']) - set(df.loc[df['date'] == date + pd.DateOffset(months=1), 'asset']))
    df_current.loc[df_current['asset'].isin(forced_sale_list), 'buy'] = df_current.loc[df_current['asset'].isin(forced_sale_list), 'sell']
    
    df_current = df_current[['asset', 'mktcap', 'exp_return', 'price', 'buy', 'sell']]
    
    return df_current

def calculate_optimal_portfolio(df: pd.DataFrame, expected_return: pd.DataFrame, portfolio: Dict, date: pd.Timestamp, ratio: float = 0.25, size: int = 1000) -> Dict:
    """
    Find the optimal trade as proportions of the current portfolio valuation. Provide the assets we want to trade along with the total explicit and implicit costs.
    """
    
    # Obtain data    
    df_current = assemble_monthly_data(df, expected_return, portfolio, date, ratio, size)
    # print(df_current[df_current['price']==0])
    A = list(df_current['asset'].unique())
    u = calculate_large_implicit_cost(portfolio[date], df_current) 
    v = calculate_small_implicit_cost(df_current)
    s = 0.0001
    t = 5/portfolio[date]['valuation'] 
    mu = dict(zip(df_current['asset'], df_current['exp_return']))
    M = 10e8
    e = 10e-8
    a = dict(zip(df_current['asset'], df_current['sell']))
    b = dict(zip(df_current['asset'], df_current['buy']))
    
    
    # Initialise Model
    m = gp.Model(f'Optimisation_{date}')
    m.Params.LogToConsole = 0
    
    # Define variables
    w = m.addVars(A, vtype = GRB.SEMICONT, lb=-1, ub=1, name='w')
    x = m.addVars(A, vtype = GRB.BINARY, name ='x')
    ce = m.addVars(A, vtype = GRB.CONTINUOUS, name='ce')
    ci = m.addVars(A, vtype = GRB.CONTINUOUS, name='ci')
    
    # Define objective
    m.setObjective(gp.quicksum(mu[j]*w[j] - ce[j] - ci[j] for j in A),GRB.MAXIMIZE)
    
    # Define constraints
    m.addConstrs( ci[j] >= u[j]*w[j]**2 for j in A)
    m.addConstrs( ci[j] >= v[j]*w[j] for j in A)
    m.addConstrs( ci[j] >= -v[j]*w[j] for j in A)
    
    m.addConstrs( ce[j] >= s*w[j] for j in A)
    m.addConstrs( ce[j] >= -s*w[j] for j in A)
    m.addConstrs( ce[j] >= t*x[j] for j in A)
    
    m.addConstrs(w[j] >= e - M*(1-x[j]) for j in A)
    m.addConstrs(w[j] <= -e + M*x[j] for j in A)
    
    m.addConstr(gp.quicksum(w[j] for j in A) == 0)
    m.addConstrs(w[j] >= a[j] for j in A)
    m.addConstrs(w[j] <= b[j] for j in A)
    
    # Solve and collect data
    m.optimize()
    
    turnover = 0
    total_cost_explicit = 0
    total_cost_implicit = 0 
    assets = {}
    
    for v in m.getVars():
        if v.varName.startswith("ce"):
            total_cost_explicit += v.x
        elif v.varName.startswith("ci"):
            total_cost_implicit += v.x
    
    valuation = portfolio[date]['valuation']
    
    for v in m.getVars():
        if v.varName.startswith("w"):
            turnover += abs(v.x)
           
            varname = v.varName
            varname = varname.strip("w[").strip("]")
            price = float(df_current.loc[df_current.asset == varname, "price"])
            
            if varname in portfolio[date]['asset'].keys():
                assets[varname] = (portfolio[date]['asset'][varname][0], price)
                if (v.x != 0) and (assets[varname][0] + np.floor(v.x * valuation/price > 0)):
                    assets[varname] = (portfolio[date]['asset'][varname][0] + np.floor(v.x * valuation/price), price)
                    # assets[varname][0] += np.floor(v.x * valuation/price)
            elif v.x > 0:
                assets[varname] = (np.floor(v.x * valuation/price), price)
    
    output_dict = {
        "turnover": turnover/2,
        "total_cost_explicit": total_cost_explicit,
        "total_cost_implicit": total_cost_implicit,
        "asset": assets
    }
    
    return output_dict


def optimise_markdowns(optimal_portfolio: Dict, mktcap_portfolio:Dict, n_dates: int = len(df['date'].unique()), ratio: float = 0.25, size: int = 1000):
    """
    Loop through the set of dates, update the target portfolio and the market cap portfolio for each date, 
    find the optimal trades and finally add the optimal values to the dictionary
    """
    for date in pd.to_datetime(df['date'].sort_values().unique())[:n_dates]:
        # filter the dataframe to the current date 
        df_current = df[df['date'] == date]
        
        # initialise current date key in optimal and mktcap portfolio
        optimal_portfolio[date] = {}
        mktcap_portfolio[date] = {}
        
        # update dividents paid
        prev_optimal_assets = optimal_portfolio[date-pd.DateOffset(months=1)]['asset'] 
        prev_mktcap_assets = mktcap_portfolio[date-pd.DateOffset(months=1)]['asset'] 
        
        optimal_div_paid, optimal_asset_value  = value_assets_and_dividends(prev_optimal_assets, df_current)
        mktcap_div_paid, mktcap_asset_value  = value_assets_and_dividends(prev_mktcap_assets, df_current)
                
        optimal_portfolio[date]['div_paid'] = optimal_div_paid
        optimal_portfolio[date]['valuation'] = optimal_asset_value + optimal_div_paid + optimal_portfolio[date - pd.DateOffset(months=1)]['cash'] 
        optimal_portfolio[date]['asset'] = optimal_portfolio[date-pd.DateOffset(months=1)]['asset'] 
        
        mktcap_portfolio[date]['div_paid'] = mktcap_div_paid
        mktcap_portfolio[date]['valuation'] = mktcap_asset_value + mktcap_div_paid + mktcap_portfolio[date - pd.DateOffset(months=1)]['cash'] 
        
        # subset to eligible assets names
        mktcap_asset_list = filter_eligible_investments(df, date, ratio, size)
        mktcap_df = df[(df['date'] == date) | (df['asset'].isin(mktcap_asset_list))]
    
        # update the assets to the preferred ones
        mktcap_portfolio[date]['asset'] = update_assets(mktcap_df, mktcap_portfolio[date]['valuation']) 
        
        # find optimal asset allocation
        optimised_outputs = calculate_optimal_portfolio(df, expected_return, optimal_portfolio, date, ratio, size)
        optimal_portfolio[date]['asset'] = optimised_outputs['asset']
        optimal_portfolio[date]['turnover'] = optimised_outputs['turnover']
        optimal_portfolio[date]['total_cost_explicit'] = optimised_outputs['total_cost_explicit']
        optimal_portfolio[date]['total_cost_implicit'] = optimised_outputs['total_cost_implicit']
        total_cost = optimised_outputs['total_cost_explicit'] + optimised_outputs['total_cost_implicit']
        
        logging.info(f" Valuation of optimal portfolio at time {date} is {optimal_portfolio[date]['valuation']}")
        logging.info(f" Valuation of no cost portfolio at time {date} is {mktcap_portfolio[date]['valuation']}")
        
        # update cash amount
        new_asset_value = sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in optimal_portfolio[date]['asset'].items()])
        optimal_portfolio[date]['cash'] = optimal_portfolio[date]['valuation'] - new_asset_value - total_cost 
        mktcap_portfolio[date]['cash'] = mktcap_portfolio[date]['valuation'] - sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in mktcap_portfolio[date]['asset'].items()]) - total_cost
    
    performance = calculate_performance(optimal_portfolio)
    logging.info(f" Performance of target portfolio is {performance}")
    mean_trade_cost = calculate_mean_trade_cost(optimal_portfolio)
    logging.info(f" Mean trade cost of target portfolio is {mean_trade_cost}")
    average_turnover = calculate_average_turnover(optimal_portfolio)
    logging.info(f" Average turnover of target portfolio is {average_turnover}")
        
    return optimal_portfolio, performance, mean_trade_cost, average_turnover
    

# TODO: 
* add comments and readability to the code for group to be able to walk through easily


In [84]:
target_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 0, 
        'cash': 0,
        'div_paid': 0,
        'asset': {'sell_asset':(10e8,1)},
        'turnover': 0,
        'total_cost_explicit': 0,
        'total_cost_implicit': 0,
    },
}

no_cost_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 10e8, # CASH+SHARES*PRICE + DIV
        'cash': 10e8,
        'div_paid': 0, 
        'asset': {}, 
    },
}

final_portfolio, performance, mean_trade_cost, average_turnover = optimise_markdowns(target_portfolio, no_cost_portfolio, n_dates=5, ratio=0.25, size=1000)

INFO:root: Valuation of optimal portfolio at time 1972-01-01 00:00:00 is 1000000000.0
INFO:root: Valuation of no cost portfolio at time 1972-01-01 00:00:00 is 1000000000.0
INFO:root: Valuation of optimal portfolio at time 1972-02-01 00:00:00 is 1045806425.9110497
INFO:root: Valuation of no cost portfolio at time 1972-02-01 00:00:00 is 995037626.2847867
INFO:root: Valuation of optimal portfolio at time 1972-03-01 00:00:00 is 1070987145.3667675
INFO:root: Valuation of no cost portfolio at time 1972-03-01 00:00:00 is 990288421.8378787
INFO:root: Valuation of optimal portfolio at time 1972-04-01 00:00:00 is 1091548042.3317919
INFO:root: Valuation of no cost portfolio at time 1972-04-01 00:00:00 is 985349043.1601152
INFO:root: Valuation of optimal portfolio at time 1972-05-01 00:00:00 is 1141173836.528845
INFO:root: Valuation of no cost portfolio at time 1972-05-01 00:00:00 is 980173265.5344257
INFO:root: Performance of target portfolio is 0.30227772517796314
INFO:root: Mean trade cost of t

In [10]:
final_portfolio[pd.to_datetime("197207", format = "%Y%m")]

{'div_paid': 21786.42664837218,
 'valuation': 1079966545.5421438,
 'asset': {"b'10233'": (245282.0, 1.39955589750912),
  "b'10321'": (164527.0, 1.23437475303064),
  "b'10874'": (358627.0, 0.9761914377585),
  "b'11105'": (351321.0, 0.999999432609623),
  "b'11308'": (26501769.0, 1.18580233160066),
  "b'11754'": (2647055.0, 1.36262250365318),
  "b'12095'": (94872.0, 1.44781086128437),
  "b'13557'": (179847.0, 1.02238881276574),
  "b'13856'": (285123.0, 1.25719517989444),
  "b'13901'": (288194.0, 1.5489514911602),
  "b'13936'": (134181.0, 0.815920346870743),
  "b'14250'": (145283.0, 1.10447587077965),
  "b'15077'": (186699.0, 0.922619186603323),
  "b'15325'": (214639.0, 1.32947858890274),
  "b'15763'": (757332.0, 1.06293756145013),
  "b'16758'": (142955.0, 1.17352779126877),
  "b'17340'": (127115.0, 1.3000728885968),
  "b'17961'": (144411.0, 1.21305886977021),
  "b'18091'": (117051.0, 2.38815655478699),
  "b'18163'": (888681.0, 1.20625048919057),
  "b'18331'": (103986.0, 0.992482095547219)